In [1]:
import os
import re
import random
import gensim
import numpy as np
import pandas as pd
import nltk
from six import iteritems
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import SpectralClustering
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import cluster

In [2]:
from Litho.nlp_funcs import *
from Litho.similarity import (check_similarity, match_lithcode, jaccard_similarity, 
                              calc_similarity_score, print_sim_compare, merge_similar_words)

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
root = os.getcwd()+'/'
file = 'reclasified.csv'

DF = pd.read_csv(root+file)

In [41]:
stopa = nltk.corpus.stopwords.words('english')

In [7]:
DF = DF[['Description']]
DF = DF.dropna(how = 'any')
Descriptions=DF.Description.tolist()
Descriptions = [str(n) for n in Descriptions]

#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in Descriptions:
    allwords_stemmed = tokenize_and_stem(i, stop) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stop)
    totalvocab_tokenized.extend([allwords_tokenized])

print(np.shape(totalvocab_tokenized),  np.shape(totalvocab_stemmed))

(130085,) (130085,)


In [8]:
###here just tokens or stemmed tokens can be used alternatively
dictionary = gensim.corpora.Dictionary(totalvocab_stemmed)
## newt two lines remove tokens contained just once in the dictionary
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(once_ids)

dictionary.compactify()
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in totalvocab_stemmed]
print('length corpus', len(corpus))

Dictionary(1361 unique tokens: ['sand', 'clay', 'drift', 'suppli', 'water']...)
length corpus 130085


In [78]:
stopw2 = ['redish', 'reddish', 'red', 'black', 'blackish', 'brown', 'brownish',
          'blue', 'blueish', 'orange', 'orangeish', 'gray', 'grey', 'grayish',
          'greyish', 'white', 'whiteish', 'purple', 'purpleish', 'yellow',
          'yellowish', 'green', 'greenish', 'light', 'very', 'pink', 'coloured', 'multicoloured',
          'dark', 'color', 'colour', 'hard', 'soft', 'water', 'supply', 'fine', 'coarse',
          'medium', 'bearing', 'pipe', 'sticky', 'tough', 'small', 'stiff', 
          'running', 'streaks', 'nominal', 'bands', 'back', 'slippery', 'loose', 
          'broken', 'fractured', 'surface']


stops= stopa+stopw2  # add the additional stopwords above

In [79]:
Namoi = 'reclasified.csv'
Data = pd.read_csv(root+Namoi)
DataSampled = Data.sample(1000)
DataDes=DataSampled.Description.tolist()
DataDes = [str(n) for n in DataDes]


# #use extend so it's a big flat list of vocab
stemmed = []
tokenized = []
for i in DataDes:
    allwords_stemmed = tokenize_and_stem(i, stops) #for each item in 'synopses', tokenize/stem
    stemmed.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stops)
    tokenized.extend([allwords_tokenized])

print(np.shape(tokenized),  np.shape(stemmed))


(1000,) (1000,)


In [80]:
%time tfidf = gensim.models.TfidfModel(corpus, normalize = True)
corpus_tfidf = tfidf[corpus]

CPU times: user 114 ms, sys: 94 µs, total: 114 ms
Wall time: 113 ms


In [74]:
tfidfSim = gensim.similarities.Similarity('/home/ignacio/jugo',corpus_tfidf, len(dictionary))

In [75]:
# tfidfSim = gensim.similarities.Similarity()
# tfidfSimMatrix =[]
simsss = []
for n, i in enumerate(tokenized):
    vec_bow = dictionary.doc2bow(i)
    vec_tfidf = tfidf[vec_bow]
    sims1 = tfidfSim[vec_tfidf]
    simsss.append(sims1)
#     print(sims1)
#     tfidfSimMatrix.append(sims1)
# tfidfDisMatrix = 1- np.array(tfidfSimMatrix) 


In [83]:
DataSampled['sim'] = simsss
X = np.vstack(DataSampled.sim)
kmeans = cluster.KMeans(n_clusters = 90).fit(X)
pred_classes = kmeans.predict(X)

print(metrics.adjusted_rand_score(DataSampled.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(DataSampled.OWN.tolist(), pred_classes.tolist()))

0.563422275581 0.651132177261


In [84]:
metrics.completeness_score(DataSampled.OWN.tolist(), pred_classes.tolist())

0.70897165896437198

In [81]:
lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

In [82]:
lsiSim = gensim.similarities.Similarity('/home/ignacio/jugo',lsi[corpus_tfidf], len(dictionary))
simsss =[]
for n in tokenized:
    vec_bow = dictionary.doc2bow(n)
    vec_lsi = lsi[vec_bow]
    sims1 = lsiSim[vec_lsi]
    simsss.append(sims1)
# lsiDisMatrix = 1- np.array(lsiSimMatrix) 

In [59]:
lda = gensim.models.LdaModel(tfidf[corpus], id2word=dictionary, num_topics=50, update_every=5, chunksize=10000, passes=10)

In [60]:
ldaSim = gensim.similarities.Similarity('/home/ignacio/jugo', lda[corpus_tfidf], len(dictionary))
simsss =[]
for n in tokenized:
    vec_bow = dictionary.doc2bow(n)
    vec_lda = lda[vec_bow]
    sims1 = ldaSim[vec_lda]
    simsss.append(sims1)